# Apply Naive Bayes

In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(e1071)

In [2]:
set.seed(1)

In [3]:
# invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150', 'TimeHeadway')
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150', 'RightLamp', 'LeftLamp', 'UpLamp')

In [4]:
valuables <- c("RoadType","CurveAverage","Speed","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed","LaneCount","Engine","PathType","SteeringAngle")

In [5]:
printf <- function(...) cat(sprintf(...))

In [16]:
CV <- function(dfx, valuables, verbose=FALSE) {        
    folds <- createFolds(dfx$flag)
    
    count <- 1
    errs <- c()
    reds <- c()
    blues <- c()
    for (ids in folds) {
        train <- dfx[-ids, ]
        test <- dfx[ids, ]  
        nb <- naiveBayes(flag ~ ., data=train, laplace=3)
        p <- predict(nb, newdata=test)
        
        if (verbose) {            
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, p)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            count <- count + 1
            printf("\n")
        }
        
        nerr <- sum((p == test$flag) == FALSE)
        errs <- c(errs, nerr / nrow(test))
    }
    
    return(c(mean(errs), mean(reds), mean(blues)))
}

In [7]:
correctVsPredict <- function(test, predictedFlags) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum((predictedRedRows$flag == 'Red') == TRUE)
    nPredictedReds <- nrow(predictedRedRows)
    printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum((predictedBlueRows$flag == 'Blue') == TRUE)
    nPredictedBlues <- nrow(predictedBlueRows)
    printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [8]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [9]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Red

In [10]:
df3 <- read.csv("../data/middle/sp4.csv", stringsAsFactors=FALSE)

In [11]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [21]:
dfx <- df3[, c(valuables, "flag")]

In [22]:
set.seed(1)

In [23]:
result <- CV(dfx, valuables, verbose=TRUE)
printRedRatios(dfx)
printf("Red: Mean correct/predict = %f\n", result[2])
printf("Blue: Mean correct/predict = %f\n", result[3])
printf("CV value: %f", result[1])

Fold1
As for Red: correct/predict = 20/26 = 0.769231
As for Blue: correct/predict = 38/78 = 0.487179

Fold2
As for Red: correct/predict = 24/35 = 0.685714
As for Blue: correct/predict = 34/70 = 0.485714

Fold3
As for Red: correct/predict = 18/30 = 0.600000
As for Blue: correct/predict = 33/74 = 0.445946

Fold4
As for Red: correct/predict = 18/31 = 0.580645
As for Blue: correct/predict = 32/73 = 0.438356

Fold5
As for Red: correct/predict = 24/36 = 0.666667
As for Blue: correct/predict = 33/68 = 0.485294

Fold6
As for Red: correct/predict = 21/34 = 0.617647
As for Blue: correct/predict = 32/70 = 0.457143

Fold7
As for Red: correct/predict = 23/29 = 0.793103
As for Blue: correct/predict = 39/76 = 0.513158

Fold8
As for Red: correct/predict = 22/28 = 0.785714
As for Blue: correct/predict = 39/76 = 0.513158

Fold9
As for Red: correct/predict = 24/38 = 0.631579
As for Blue: correct/predict = 31/66 = 0.469697

Fold10
As for Red: correct/predict = 14/25 = 0.560000
As for Blue: correct/predict